In [ ]:
import numpy as np
import matplotlib

matplotlib.use("agg")

import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

golden_mean = (np.sqrt(5) - 1) / 2  # Aesthetic ratio
fig_width_pt = 246.0  # Columnwidth
inches_per_pt = 1 / 72.27  # Convert pt to inches
fig_width = fig_width_pt * inches_per_pt
fig_height = fig_width * golden_mean  # height in inches
fig_size = [fig_width, fig_height]

params = {
    "backend": "ps",
    "axes.labelsize": 13,
    "font.size": 13,
    "legend.fontsize": 10,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "text.usetex": True,
    "figure.figsize": fig_size,
    "font.family": "serif",
    "font.serif": "Computer Modern Roman",
    "legend.frameon": True,
    "savefig.dpi": 300,
}

plt.rcParams.update(params)
plt.rc("text.latex", preamble=[r"\usepackage{xfrac}", r"\usepackage{siunitx}"])

In [ ]:
np.random.seed(1)
xs = np.sort(np.random.uniform(-1, 1, 3))
errs = np.abs(np.random.randn(3))
ys = xs ** 3
means = lambda x: np.convolve(x, np.ones(2) / 2, mode="valid")
xs_means = means(xs)
ys_means = means(ys)

fig, ax = plt.subplots()
plt.scatter(xs, ys, c="k")
ax.errorbar(xs, ys, errs, capsize=5, c="k")
ax.annotate(
    s=r"$L_{1,2} = \sqrt{\Delta x^2 + \Delta \bar{y}^2}$",
    xy=(np.mean([xs[0], xs[1], xs[1]]), np.mean([ys[0], ys[1], ys[1]])),
    xytext=(xs_means[0], ys_means[0] + 1),
    arrowprops=dict(arrowstyle="->"),
    ha="center",
)

for i, (x, y, err) in enumerate(zip(xs, ys, errs)):
    err_str = fr'${{\sigma}}_{{\bar {{y}}_{i+1}}}$'
    ax.annotate(
        s=err_str,
        xy=(x, y + err / 2),
        xytext=(x + 0.1, y + err + 0.5),
        arrowprops=dict(arrowstyle="->"),
        ha="center",
    )

    ax.annotate(
        s=fr"$x_{i+1}, \bar{{y}}_{i+1}$",
        xy=(x, y),
        xytext=(x + 0.1, y - 0.5),
        arrowprops=dict(arrowstyle="->"),
        ha="center",
    )


ax.scatter(xs, ys, c="green", s=5, zorder=5, label="more seeds")
ax.scatter(xs_means, ys_means, c="red", s=5, zorder=5, label="new point")
ax.legend()

ax.text(
    x=0.5,
    y=0.0,
    s=(
        r"$\textrm{if}\; \max{(L_{i,i+1})} > \textrm{average\_priority} \cdot \max{\sigma_{\bar{y}_{i}}} \rightarrow,\;\textrm{add new point}$"
        "\n"
        r"$\textrm{if}\; \max{(L_{i,i+1})} < \textrm{average\_priority} \cdot \max{\sigma_{\bar{y}_{i}}} \rightarrow,\;\textrm{add new seeds}$"
    ),
    horizontalalignment="center",
    verticalalignment="center",
    transform=ax.transAxes,
)
ax.set_title("AverageLearner1D")
ax.axis("off")
plt.show()

In [ ]:
np.random.seed(1)
xs = np.array([0.1, 0.3, 0.35, 0.45])
f = lambda x: x**3
ys = f(xs)
means = lambda x: np.convolve(x, np.ones(2) / 2, mode="valid")
xs_means = means(xs)
ys_means = means(ys)

fig, ax = plt.subplots(figsize=fig_size)
ax.scatter(xs, ys, c="k")
ax.plot(xs, ys, c="k")
# ax.scatter()
ax.annotate(
    s=r"$L_{1,2} = \sqrt{\Delta x^2 + \Delta y^2}$",
    xy=(np.mean([xs[0], xs[1]]), np.mean([ys[0], ys[1]])),
    xytext=(xs[0]+0.05, ys[0] - 0.05),
    arrowprops=dict(arrowstyle="->"),
    ha="center",
    zorder=10,
)

for i, (x, y) in enumerate(zip(xs, ys)):
    sign = [1, -1][i % 2]
    ax.annotate(
        s=fr"$x_{i+1}, y_{i+1}$",
        xy=(x, y),
        xytext=(x + 0.01, y + sign * 0.04),
        arrowprops=dict(arrowstyle="->"),
        ha="center",
    )
    
ax.scatter(xs, ys, c="green", s=5, zorder=5, label="existing data")
losses = np.hypot(xs[1:] - xs[:-1], ys[1:] - ys[:-1])
ax.scatter(xs_means, ys_means, c="red", s=300*losses, zorder=8, label="candidate points")
xs_dense = np.linspace(xs[0], xs[-1], 400)
ax.plot(xs_dense, f(xs_dense), alpha=0.3, zorder=7, label="function")

ax.legend()
ax.axis("off")
plt.savefig("figures/loss_1D.pdf", bbox_inches="tight", transparent=True)
plt.show()

In [ ]:
import adaptive
adaptive.notebook_extension()

In [ ]:
def f(x, offset=0.12312):
    a = 0.01
    return x + a**2 / (a**2 + (x - offset)**2)

learner = adaptive.Learner1D(f, bounds=(-1, 1))
adaptive.runner.simple(learner, goal=lambda l: l.npoints > 100)


In [ ]:
xs, ys = zip(*learner.data.items())

In [ ]:
fig, ax = plt.subplots()
for i in range(1, len(xs)):
    if i % 10 != 0:
        continue
    alpha = np.linspace(0.2, 1, 101)[i]
    offset = i / len(xs)
    xs_part, ys_part = xs[:i], ys[:i]
    xs_part, ys_part = zip(*sorted(zip(xs_part, ys_part)))
    ax.plot(xs_part, offset + np.array(ys_part), alpha=alpha, c='grey', lw=0.5)

plt.show()

In [ ]:
xs_part, ys_part